In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Data has been found from Kaggle
# https://www.kaggle.com/datasnaek/youtube-new

# YouTube video like prediction

In [15]:
import kaggle

DATASET_NAME = 'datasnaek/youtube-new'
DATABASE_FOLDER = os.path.join('datasets', 'youtube')
US_VIDEOS = os.path.join(DATABASE_FOLDER, 'USvideos.csv')

def fetch_data(name=DATASET_NAME, path=DATABASE_FOLDER):
    '''
        Downloads the data with the kaggle command
    '''
    !kaggle datasets download -d $name -p $path
    !unzip $path/* $path
    

fetch_data(DATASET_NAME, DATABASE_FOLDER)    

data = pd.read_csv(US_VIDEOS, parse_dates=['trending_date', 'publish_time'])
data.head()
    
    


youtube-new.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  datasets/youtube/youtube-new.zip
caution: filename not matched:  datasets/youtube


FileNotFoundError: [Errno 2] No such file or directory: 'datasets/youtube/USvideos.csv'

In [ ]:
data = data.drop(['description', 'thumbnail_link', 'title', 'tags', 'channel_title', 'video_id'], axis=1)
data.head()


In [ ]:
data_analyse = data.copy()

data_analyse.hist(bins=10, figsize=(12, 8))

In [ ]:
corr_matrix = data_analyse.corr()
corr_matrix['views'].sort_values(ascending=False)


In [ ]:
from pandas.plotting import scatter_matrix


corellation_attributes = ['views', 'likes', 'comment_count', 'category_id']
scatter_matrix(data_analyse[corellation_attributes], figsize=(12, 8))

# It can be observed that likes are very correllated to views, comments.
# Some categories have definitively more success than others

In [ ]:
# The difference between the publish time and the trending_date may also be interesting

data_analyse['trending_date'] = pd.to_datetime(data_analyse['trending_date'], format='%y.%d.%m')

data_analyse['publish_time'] = data_analyse['publish_time'].dt.date
data_analyse['trending_date'] = data_analyse['trending_date'].dt.date

data_analyse['trending_difference'] = (data_analyse['trending_date'] - data_analyse['publish_time']).dt.days

In [ ]:
# No apparent linear corellation with the trending_difference

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

print(data_analyse.describe())
num_attributes = list(data_analyse.drop('category_id'))
cat_attributes = ['category_id']†

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),  # Must be transformer (fit_transform must be defined)
    ('std_scaler', StandardScaler())  # Doesn't have to be a transformer
])

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attributes)
    ('cat', OneHotEncoder(), cat_attributes)
])

data = data_analyse.copy()

data['trending_date'] = pd.to_datetime(data['trending_date'], format='%y.%d.%m')
data['trending_difference'] = (data['trending_date'].dt.date - data['publish_time'].dt.date).dt.days

data_analyse['publish_time'] = data_analyse['publish_time'].dt.days
data_analyse['trending_date'] = data_analyse['trending_date'].dt.days
